In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Reading from Sockets")
    .master("local[*]")
    .getOrCreate()
    
)

spark

In [17]:
# READ INPUT DATA

df_raw = spark.readStream.format("socket").option("host","localhost").option("port", "9999").load()

In [18]:
df_raw.printSchema()

root
 |-- value: string (nullable = true)



In [19]:
#Split the line into words
from pyspark.sql.functions import split
df_words = df_raw.withColumn("words", split("value"," "))

In [20]:
# Explode the list of words
from pyspark.sql.functions import explode

df_explode = df_words.withColumn("word", explode("words")).drop("value","words")

In [21]:
# Aggregate the words to generate count
from pyspark.sql.functions import count, lit
df_agg = df_explode.groupBy("word").agg(count(lit(1)).alias("cnt"))

In [ ]:
#Write the output to console streaming

df_agg.writeStream.format("console").outputMode("complete").start().awaitTermination()